In [1]:
import geemap
import pandas as pd
import numpy as np
import pdb
from IPython.display import display
import ee
import os

In [2]:
ee.Initialize()

In [3]:
iowa = geemap.shp_to_ee('F:/US states/iowa.shp')

In [4]:
dem = ee.ImageCollection('USGS/3DEP/1m') \
    .filterBounds(iowa) \
    .mosaic()

hillshade_iowa = ee.Terrain.hillshade(dem.select('elevation'))

dem_vis = {
  'min': 0,
  'max': 3000,
  'palette': [
    '3ae237', 'b5e22e', 'd6e21f', 'fff705', 'ffd611', 'ffb613', 'ff8b13',
    'ff6e08', 'ff500d', 'ff0000', 'de0101', 'c21301', '0602ff', '235cb1',
    '307ef3', '269db1', '30c8e2', '32d3ef', '3be285', '3ff38f', '86e26f'
  ]
}

In [5]:
iowa_mosaic_2010 = ee.ImageCollection('USDA/NAIP/DOQQ') \
    .filterBounds(iowa) \
    .filterDate('2010-01-01', '2010-12-31') \
    .mosaic() \
    .addBands(dem.select('elevation'))


nlcd_iowa = ee.ImageCollection("USGS/NLCD_RELEASES/2021_REL/NLCD") \
    .filterBounds(iowa) \
    .filterDate('2001-01-01', '2021-12-31') \
    .select('landcover') \
    .mosaic() \
    .clip(iowa)

buffer_points = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_areas/buffer2.shp') #no overlaps

# contour_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips/contour_buffer.shp')
# grassed_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips/grassed_buffer.shp')
# dam_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips/dam_buffer.shp')
# strip_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips/strip_buffer.shp')
# terrace_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips/terrace_buffer.shp')
# basin_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips/basin_buffer.shp')

#Switched to bufferclips2 folder - no overlaps, 100 polygons
contour_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips2/contour_buffer.shp')
grassed_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips2/grassed_buffer.shp')
dam_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips2/dam_buffer.shp')
strip_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips2/strip_buffer.shp')
terrace_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips2/terrace_buffer.shp')
basin_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips2/basin_buffer.shp')

#Create a mask for BMP clips:

def maskInside(image, geometry):
    mask = ee.Image.constant(1).clip(geometry).mask().Not()
    return image.updateMask(mask)

def maskFarmland(image):
    mask = image.eq(71).Or(image.eq(81)).Or(image.eq(82))
    return image.updateMask(mask)


# Combine all feature geometries into a single multipolygon
all_features = contour_clip.merge(grassed_clip).merge(dam_clip).merge(strip_clip).merge(terrace_clip).merge(basin_clip)
all_multipolygon = all_features.geometry()

# Apply the  mask to the NLCD image
# nlcd_filtered = maskInside(nlcd_mask, all_multipolygon)

nlcd_filtered = maskInside(maskFarmland(nlcd_iowa), all_multipolygon)

In [6]:
# Create a map
Map = geemap.Map(center=[40.6, -94], zoom=10)

# Map.addLayer(nlcd_iowa,{},'NLCD')
# Map.addLayer(nlcd_mask, {}, 'NLCD Mask')

iowa_masked = iowa_mosaic_2010.updateMask(nlcd_filtered)

# Map.addLayer(nlcd_filtered, {}, 'Filtered NLCD')
Map.addLayer(iowa_masked, {}, 'Masked')

Map.addLayer(buffer_points, {}, 'buffer points')

# Map.addLayer(dem, dem_vis, 'elevation')
# Map.addLayer(hillshade_iowa)

Map

Map(center=[40.6, -94], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(…